# Get from Address to Model

In [1]:
import urllib
import requests
import json

## Geo-Admin API
Access of the API

In [2]:
#Definition of parameters
text = input("Please enter an address: ") #Input of the address to search for
base_url="https://api3.geo.admin.ch/rest/services/api/SearchServer?"
parameters = {"searchText": text, "origins": "address", "type": "locations",}

#Request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

data = json.loads(r.content)
print(data)

{'results': [{'attrs': {'detail': 'widmergasse 19 8466 truellikon 40 truellikon ch zh', 'featureId': '12870_0', 'geom_quadindex': '012232210032122231000', 'geom_st_box2d': 'BOX(694347.29027415 276686.7814323578,694347.29027415 276686.7814323578)', 'label': 'Widmergasse 19 <b>8466 Trüllikon</b>', 'lat': 47.63404846191406, 'lon': 8.694016456604004, 'num': 19, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276686.78125, 'y': 694347.3125, 'zoomlevel': 10}, 'id': 7251, 'weight': 1}, {'attrs': {'detail': 'widmergasse 19.1 8466 truellikon 40 truellikon ch zh', 'featureId': '210226908_0', 'geom_quadindex': '012232210032122310321', 'geom_st_box2d': 'BOX(694360.3364026649 276691.3415635696,694360.3364026649 276691.3415635696)', 'label': 'Widmergasse 19.1 <b>8466 Trüllikon</b>', 'lat': 47.634090423583984, 'lon': 8.694190979003906, 'num': 191, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276691.34375, 'y': 694360.3125, 'zoomlevel': 10}, 'id': 2190619, 'weight': 8}, {'attrs': {'

Geographic display of returned results

In [3]:
import folium

# Create a map centered at the first item's latitude and longitude
map_center = [data['results'][0]['attrs']['lat'], data['results'][0]['attrs']['lon']]
m = folium.Map(location=map_center, zoom_start=20)

# Add markers for each item with annotations
for i, result in enumerate(data['results']):
    lat = result['attrs']['lat']
    lon = result['attrs']['lon']
    label = f"Index: {i}"
    folium.Marker([lat, lon], popup=label).add_to(m)

# Display the map
m


In [4]:
#Select the desired item
index = int(input("Please enter the index of the item you want to select: "))
selected_item = data['results'][index]
print(selected_item)


{'attrs': {'detail': 'widmergasse 19 8466 truellikon 40 truellikon ch zh', 'featureId': '12870_0', 'geom_quadindex': '012232210032122231000', 'geom_st_box2d': 'BOX(694347.29027415 276686.7814323578,694347.29027415 276686.7814323578)', 'label': 'Widmergasse 19 <b>8466 Trüllikon</b>', 'lat': 47.63404846191406, 'lon': 8.694016456604004, 'num': 19, 'objectclass': '', 'origin': 'address', 'rank': 7, 'x': 276686.78125, 'y': 694347.3125, 'zoomlevel': 10}, 'id': 7251, 'weight': 1}


## KML files
Check KML files for desired building

In [5]:
import os
import re
import math

# Define the folder path
folder_path = "swissBUILDINGS3D"

# Get the latitude and longitude from the selected_item
lat = float(selected_item['attrs']['lat'])
lon = float(selected_item['attrs']['lon'])
print(f"Latitude: {lat}", f"Longitude: {lon}")

# Define the calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth radius in kilometers

    return distance

# Search for KML files in the folder
kml_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".kml"):
            kml_files.append(os.path.join(root, file))

print(f"Found {len(kml_files)} KML files", kml_files)

# Define the tolerance in kilometers
tolerance = 0.01

# List to store the filtered KML data
filtered_kml_data = []

# Iterate through the KML files
for kml_file in kml_files:
    with open(kml_file, 'r') as file:
        kml_content = file.read()
        
        # Extract the latitude and longitude values from the KML content
        latitudes = re.findall(r'<latitude>(.*?)</latitude>', kml_content)
        longitudes = re.findall(r'<longitude>(.*?)</longitude>', kml_content)
        hrefs = re.findall(r'<href>(.*?)</href>', kml_content)
        
        # Convert the latitude and longitude values to float
        latitudes = [float(lat) for lat in latitudes]
        longitudes = [float(lon) for lon in longitudes]
        
        # Check if any entry is within the tolerance
        for i in range(len(latitudes)):
            distance = calculate_distance(lat, lon, latitudes[i], longitudes[i])
            if distance <= tolerance:
                filtered_kml_data.append((kml_file, latitudes[i], longitudes[i],  hrefs[i]))

# Print the filtered KML data
for data in filtered_kml_data:
    print(f"KML File: {data[0]}, Latitude: {data[1]}, Longitude: {data[2]}, href: {data[3]}")



Latitude: 47.63404846191406 Longitude: 8.694016456604004
Found 6 KML files ['swissBUILDINGS3D\\1031-44\\1031-44.kml', 'swissBUILDINGS3D\\1032-33\\1032-33.kml', 'swissBUILDINGS3D\\1032-34\\1032-34.kml', 'swissBUILDINGS3D\\1051-22\\1051-22.kml', 'swissBUILDINGS3D\\1052-11\\1052-11.kml', 'swissBUILDINGS3D\\1052-12\\1052-12.kml']
KML File: swissBUILDINGS3D\1052-11\1052-11.kml, Latitude: 47.634047498746725, Longitude: 8.694078211780825, href: models/model_2045.dae


## Get model data from kml

In [6]:
import collada

for data in filtered_kml_data:
    kml_file = data[0]
    lat = data[1]
    lon = data[2]
    href = data[3]
    
    # Load the COLLADA file
    coll = collada.Collada("swissBUILDINGS3D\\1052-11\\models\\model_2045.dae")
    print ("swissBUILDINGS3D\\1052-11\\models\\model_2045.dae")
    # Extract the geometry
    print(coll)
    print(coll.geometries)
    print(coll.geometries[0].primitives)
    for i in coll.geometries[0].primitives[0]:
        print(i)


swissBUILDINGS3D\1052-11\models\model_2045.dae
<Collada geometries=4>
[<Geometry id=g_0, 1 primitives>, <Geometry id=g_1, 1 primitives>, <Geometry id=g_2, 1 primitives>, <Geometry id=g_3, 1 primitives>]
[<TriangleSet length=10>]
<Triangle ([ -3.1614728   3.1393507 477.35     ], [  0.6268737   8.523992  476.011    ], [ -4.5226707   6.318286  476.011    ], "material0")>
<Triangle ([  3.954176   -1.1537193 479.961    ], [  0.6268737   8.523992  476.011    ], [ -3.1614728   3.1393507 477.35     ], "material0")>
<Triangle ([  4.6439295  -0.8587421 479.961    ], [  0.6268737   8.523992  476.011    ], [  3.954176   -1.1537193 479.961    ], "material0")>
<Triangle ([ -3.0441387  -4.1518803 479.961    ], [ -3.1614728   3.1393507 477.35     ], [ -6.389766    1.7559415 477.35     ], "material0")>
<Triangle ([  3.954176   -1.1537193 479.961    ], [ -3.1614728   3.1393507 477.35     ], [ -3.0441387  -4.1518803 479.961    ], "material0")>
<Triangle ([ -3.0441387  -4.1518803 479.961    ], [ -6.389766

In [7]:
import numpy as np
from ipygany import PolyMesh

meshes=[]

for i in range(len(coll.geometries)):
    
    vertices = []
    faces = []

    for triangle in coll.geometries[i].primitives[0]:
        # Extract the vertices of the triangle
        v1, v2, v3 = triangle.vertices
        
        # Add the vertices to the vertices list
        vertices.append(v1)
        vertices.append(v2)
        vertices.append(v3)
        
        # Calculate the indices of the vertices in the faces list
        index1 = len(vertices) - 3
        index2 = len(vertices) - 2
        index3 = len(vertices) - 1
        
        # Add the indices to the faces list
        faces.append([index1, index2, index3])

    # Convert the vertices and faces lists to numpy arrays
    vertices = np.array(vertices)
    faces = np.array(faces)

    # Create the PolyMesh
    mesh = PolyMesh(vertices=vertices, faces=faces)
    meshes.append(mesh)


In [8]:
from ipygany import Scene

#Draws the geometry
scene = Scene([meshes[3]])
scene

Scene(children=[PolyMesh(data=[], triangle_indices=array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  1…

## Upload images from the building